In [1]:
import plotly
from plotly import plotly as py
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import snowflake.connector
import pandas as pd
import numpy as np
import snowflake
import getpass
import pprint
from snowflake.connector import DictCursor
from datetime import datetime
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF
import plotly.figure_factory as ff

CONNECTION_PARAMS = {
    "user":None,
    "password":None,
    "account":'snagajob',
    "authenticator":"https://snagajob.okta.com/",
    "database":"PROD_SAJ_SHARE",
    "warehouse":"prod_wh",
    "schema":"DBO"
}

CONNECTION_PARAMS["user"] = input("Your Snowflake username: ")
CONNECTION_PARAMS["password"] = getpass.getpass("Your Snowflake password: ")

# Establish connection to plotly server
plotly_username = input("Your Plotly username: ")
plotly_api_key = getpass.getpass("Your Plotly API Key: ")
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)

Your Snowflake username: nazih.bissat@snagajob.com
Your Snowflake password: ········
Your Plotly username: nbissat
Your Plotly API Key: ········


In [18]:
# UNFULFILLMENT RATE ANALYSIS

date_from = input("The date to start analysis from (YYYYMMDD): ")

query1_p1 = """
select DIMDATEKEY,
count(case when (POSTINGID is not NULL and JOBSEEKERID is NULL) then 1 end) as
      unregistered_valid_query_return,
count(case when (POSTINGID is not NULL and JOBSEEKERID is not NULL)
      then 1 end) as registered_valid_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is NULL and PAGENUMBER <= 1)
      then 1 end) as unregistered_null_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is not NULL and PAGENUMBER <= 1)
      then 1 end) as registered_null_query_return
from EVENT.EVENTS where EVENTTYPE='Search' and DIMDATEKEY >=
"""

query1_p2 = """
 and Placement = 'Keyword Search' group by DIMDATEKEY order by DIMDATEKEY;
"""

query1 = query1_p1 + str(date_from) + query1_p2

The date to start analysis from (YYYYMMDD): 20180311


In [19]:
col_names = ['DIMDATEKEY', 'UNREGISTERED_VALID_QUERY_RETURN', 'REGISTERED_VALID_QUERY_RETURN',
                          'UNREGISTERED_NULL_QUERY_RETURN', 'REGISTERED_NULL_QUERY_RETURN']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query1) as results:
            df = pd.concat([pd.DataFrame([r], columns=col_names) for r in results],
                           ignore_index=True)

In [20]:
# Converting date keys into dates
def num_to_date(num):
    s = str(num)
    return datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8]))

df['DIMDATEKEY'] = df['DIMDATEKEY'].map(lambda a: num_to_date(a))

# Calculating unfullfilment ratios for registered and unregistered users
df['REGISTERED_FULFILLMENT_RATE'] = 100 * df['REGISTERED_NULL_QUERY_RETURN'] / (df['REGISTERED_VALID_QUERY_RETURN'] 
                                                                                + df['REGISTERED_NULL_QUERY_RETURN'])

df['UNREGISTERED_FULFILLMENT_RATE'] = 100 * df['UNREGISTERED_NULL_QUERY_RETURN'] / (df['UNREGISTERED_VALID_QUERY_RETURN']
                                                                                    + df['UNREGISTERED_NULL_QUERY_RETURN'])

df.head()

,DIMDATEKEY,UNREGISTERED_VALID_QUERY_RETURN,REGISTERED_VALID_QUERY_RETURN,UNREGISTERED_NULL_QUERY_RETURN,REGISTERED_NULL_QUERY_RETURN,REGISTERED_FULFILLMENT_RATE,UNREGISTERED_FULFILLMENT_RATE
0,2018-03-11,1164721,159648,21274,6031,3.640172,1.793768
1,2018-03-12,1369227,256741,40146,12600,4.678085,2.848501
2,2018-03-13,1865526,254835,38991,10315,3.890251,2.047291
3,2018-03-14,1646983,243692,41606,9892,3.900877,2.463951
4,2018-03-15,1598975,231892,32422,9442,3.912420,1.987376


In [21]:
df['MOVING_AVERAGE_REGISTERED'] = 0
df['MOVING_SD_REGISTERED'] = 0

df['MOVING_AVERAGE_UNREGISTERED'] = 0
df['MOVING_SD_UNREGISTERED'] = 0

for i in range(0, 14):
    df.loc[i, 'MOVING_AVERAGE_REGISTERED'] = df.loc[i, 'REGISTERED_FULFILLMENT_RATE']
    df.loc[i, 'MOVING_AVERAGE_UNREGISTERED'] = df.loc[i, 'UNREGISTERED_FULFILLMENT_RATE']

for i in range(14, len(df)):
    df.loc[i, 'MOVING_AVERAGE_REGISTERED'] = np.mean(df.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df.loc[i, 'MOVING_SD_REGISTERED'] = np.std(df.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df.loc[i, 'MOVING_AVERAGE_UNREGISTERED'] = np.mean(df.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())
    df.loc[i, 'MOVING_SD_UNREGISTERED'] = np.std(df.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())

error_registered_up = np.array((df['MOVING_AVERAGE_REGISTERED'] + 2 * df['MOVING_SD_REGISTERED']).tolist())

error_registered_down = np.array((df['MOVING_AVERAGE_REGISTERED'] - 2 * df['MOVING_SD_REGISTERED']).tolist())

error_unregistered_up = np.array((df['MOVING_AVERAGE_UNREGISTERED'] + 2 * df['MOVING_SD_UNREGISTERED']).tolist())

error_unregistered_down = np.array((df['MOVING_AVERAGE_UNREGISTERED'] - 2 * df['MOVING_SD_UNREGISTERED']).tolist())

In [22]:
dates = df['DIMDATEKEY'].tolist()
avg_dates = dates[14:len(dates)]

trace0 = go.Scatter(
    x=avg_dates,
    y=error_registered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace1 = go.Scatter(
    x=avg_dates,
    y=error_registered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace2 = go.Scatter(
    x=df['DIMDATEKEY'].tolist(),
    y=df['REGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Registered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)

trace3 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace4 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace5 = go.Scatter(
    x=df['DIMDATEKEY'].tolist(),
    y=df['UNREGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Unregistered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)

layout = go.Layout(
    title='Unfulfillment Rate / User Registration Analysis',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Unfulfillment Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)

py.iplot(fig, filename = 'Unfulfillment rate analysis')

In [23]:
# UNFULFILLMENT RATE ANALYSIS FOR MOBILE BROWSER

query1_mobile_p1 = """
select DIMDATEKEY,
count(case when (POSTINGID is not NULL and JOBSEEKERID is NULL) then 1 end) as
      unregistered_valid_query_return,
count(case when (POSTINGID is not NULL and JOBSEEKERID is not NULL)
      then 1 end) as registered_valid_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is NULL and PAGENUMBER <= 1)
      then 1 end) as unregistered_null_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is not NULL and PAGENUMBER <= 1)
      then 1 end) as registered_null_query_return
from EVENT.EVENTS where EVENTTYPE='Search' and DIMDATEKEY >=
"""

query1_mobile_p2 = """
 and Placement = 'Keyword Search' and SOURCE = 'Sommersault Mobile' 
 group by DIMDATEKEY order by DIMDATEKEY;
"""

query1_mobile = query1_mobile_p1 + str(date_from) + query1_mobile_p2

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query1_mobile) as results:
            df_mobile = pd.concat([pd.DataFrame([r], columns=col_names) for r in results],
                           ignore_index=True)
            
df_mobile['DIMDATEKEY'] = df_mobile['DIMDATEKEY'].map(lambda a: num_to_date(a))

# Calculating unfullfilment ratios for registered and unregistered users
df_mobile['REGISTERED_FULFILLMENT_RATE'] = 100 * df_mobile['REGISTERED_NULL_QUERY_RETURN'] / (df_mobile['REGISTERED_VALID_QUERY_RETURN'] 
                                                                                + df_mobile['REGISTERED_NULL_QUERY_RETURN'])

df_mobile['UNREGISTERED_FULFILLMENT_RATE'] = 100 * df_mobile['UNREGISTERED_NULL_QUERY_RETURN'] / (df_mobile['UNREGISTERED_VALID_QUERY_RETURN']
                                                                                    + df_mobile['UNREGISTERED_NULL_QUERY_RETURN'])

df_mobile['MOVING_AVERAGE_REGISTERED'] = 0
df_mobile['MOVING_SD_REGISTERED'] = 0

df_mobile['MOVING_AVERAGE_UNREGISTERED'] = 0
df_mobile['MOVING_SD_UNREGISTERED'] = 0

for i in range(14, len(df_mobile)):
    df_mobile.loc[i, 'MOVING_AVERAGE_REGISTERED'] = np.mean(df_mobile.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df_mobile.loc[i, 'MOVING_SD_REGISTERED'] = np.std(df_mobile.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df_mobile.loc[i, 'MOVING_AVERAGE_UNREGISTERED'] = np.mean(df_mobile.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())
    df_mobile.loc[i, 'MOVING_SD_UNREGISTERED'] = np.std(df_mobile.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())

error_registered_up = np.array((df_mobile['MOVING_AVERAGE_REGISTERED'] + 2 * df_mobile['MOVING_SD_REGISTERED']).tolist())

error_registered_down = np.array((df_mobile['MOVING_AVERAGE_REGISTERED'] - 2 * df_mobile['MOVING_SD_REGISTERED']).tolist())

error_unregistered_up = np.array((df_mobile['MOVING_AVERAGE_UNREGISTERED'] + 2 * df_mobile['MOVING_SD_UNREGISTERED']).tolist())

error_unregistered_down = np.array((df_mobile['MOVING_AVERAGE_UNREGISTERED'] - 2 * df_mobile['MOVING_SD_UNREGISTERED']).tolist())

trace0 = go.Scatter(
    x=avg_dates,
    y=error_registered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace1 = go.Scatter(
    x=avg_dates,
    y=error_registered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace2 = go.Scatter(
    x=dates,
    y=df_mobile['REGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Registered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)

trace3 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace4 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace5 = go.Scatter(
    x=dates,
    y=df_mobile['UNREGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Unregistered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)

layout = go.Layout(
    title='Mobile Unfulfillment Rate / User Registration Analysis',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Mobile Unfulfillment Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)

py.iplot(fig, filename = 'Mobile Unfulfillment rate analysis')

In [24]:
# UNFULFILLMENT RATE ANALYSIS FOR DESKTOP BROWSER

query1_desktop_p1 = """
select DIMDATEKEY,
count(case when (POSTINGID is not NULL and JOBSEEKERID is NULL) then 1 end) as
      unregistered_valid_query_return,
count(case when (POSTINGID is not NULL and JOBSEEKERID is not NULL)
      then 1 end) as registered_valid_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is NULL and PAGENUMBER <= 1)
      then 1 end) as unregistered_null_query_return,
count(case when (POSTINGID is NULL and JOBSEEKERID is not NULL and PAGENUMBER <= 1)
      then 1 end) as registered_null_query_return
from EVENT.EVENTS where EVENTTYPE='Search' and DIMDATEKEY >=
"""

query1_desktop_p2 = """
 and Placement = 'Keyword Search' and SOURCE = 'Sommersault' 
 group by DIMDATEKEY order by DIMDATEKEY;
"""

query1_desktop = query1_desktop_p1 + str(date_from) + query1_desktop_p2

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query1_desktop) as results:
            df_desktop = pd.concat([pd.DataFrame([r], columns=col_names) for r in results],
                           ignore_index=True)
            
df_desktop['DIMDATEKEY'] = df_desktop['DIMDATEKEY'].map(lambda a: num_to_date(a))

# Calculating unfullfilment ratios for registered and unregistered users
df_desktop['REGISTERED_FULFILLMENT_RATE'] = 100 * df_desktop['REGISTERED_NULL_QUERY_RETURN'] / (df_desktop['REGISTERED_VALID_QUERY_RETURN'] 
                                                                                + df_desktop['REGISTERED_NULL_QUERY_RETURN'])

df_desktop['UNREGISTERED_FULFILLMENT_RATE'] = 100 * df_desktop['UNREGISTERED_NULL_QUERY_RETURN'] / (df_desktop['UNREGISTERED_VALID_QUERY_RETURN']
                                                                                    + df_desktop['UNREGISTERED_NULL_QUERY_RETURN'])

df_desktop['MOVING_AVERAGE_REGISTERED'] = 0
df_desktop['MOVING_SD_REGISTERED'] = 0

df_desktop['MOVING_AVERAGE_UNREGISTERED'] = 0
df_desktop['MOVING_SD_UNREGISTERED'] = 0

for i in range(14, len(df_desktop)):
    df_desktop.loc[i, 'MOVING_AVERAGE_REGISTERED'] = np.mean(df_desktop.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df_desktop.loc[i, 'MOVING_SD_REGISTERED'] = np.std(df_desktop.loc[(i-14):i, 'REGISTERED_FULFILLMENT_RATE'].tolist())
    df_desktop.loc[i, 'MOVING_AVERAGE_UNREGISTERED'] = np.mean(df_desktop.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())
    df_desktop.loc[i, 'MOVING_SD_UNREGISTERED'] = np.std(df_desktop.loc[(i-14):i, 'UNREGISTERED_FULFILLMENT_RATE'].tolist())

error_registered_up = np.array((df_desktop['MOVING_AVERAGE_REGISTERED'] + 2 * df_desktop['MOVING_SD_REGISTERED']).tolist())

error_registered_down = np.array((df_desktop['MOVING_AVERAGE_REGISTERED'] - 2 * df_desktop['MOVING_SD_REGISTERED']).tolist())

error_unregistered_up = np.array((df_desktop['MOVING_AVERAGE_UNREGISTERED'] + 2 * df_desktop['MOVING_SD_UNREGISTERED']).tolist())

error_unregistered_down = np.array((df_desktop['MOVING_AVERAGE_UNREGISTERED'] - 2 * df_desktop['MOVING_SD_UNREGISTERED']).tolist())

trace0 = go.Scatter(
    x=avg_dates,
    y=error_registered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace1 = go.Scatter(
    x=avg_dates,
    y=error_registered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)
trace2 = go.Scatter(
    x=dates,
    y=df_desktop['REGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Registered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(20, 20, 200)',
    )
)

trace3 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_down[14:len(dates)],
    name='Lower bound',
    fill=None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace4 = go.Scatter(
    x=avg_dates,
    y=error_unregistered_up[14:len(dates)],
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)
trace5 = go.Scatter(
    x=dates,
    y=df_desktop['UNREGISTERED_FULFILLMENT_RATE'].tolist(),
    name='Unregistered Users',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(200, 200, 20)',
    )
)

layout = go.Layout(
    title='Desktop Unfulfillment Rate / User Registration Analysis',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Desktop Unfulfillment Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5], layout=layout)

py.iplot(fig, filename = 'Desktop Unfulfillment rate analysis')

In [11]:
# SEARCH/PLATFORM ANALYSIS

date_from = input("The starting date of the analysis (YYYYMMDD): ")

query2_p1 = """
select DIMDATEKEY,
    count(case when SOURCE = 'Sommersault' then 1 end) as sommersault,
    count(case when SOURCE = 'Sommersault Mobile' then 1 end) as sommersault_mobile,
    count(case when SOURCE = 'Sommersault iOS' then 1 end) as sommersault_ios,
    count(case when SOURCE = 'Career Pages' then 1 end) as career_pages
from EVENT.EVENTS where EVENTTYPE='Search' and Placement = 'Keyword Search' and DIMDATEKEY >=
"""

query2_p2 = """
 group by DIMDATEKEY
order by DIMDATEKEY;
"""

query2 = query2_p1 + str(date_from) + query2_p2

col_names2 = ['DIMDATEKEY', 'SOMMERSAULT', 'SOMMERSAULT_MOBILE', 'CAREER_PAGES', 'SOMMERSAULT_IOS']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query2) as results:
            df2 = pd.concat([pd.DataFrame([r], columns=col_names2) for r in results],
                           ignore_index=True)

In [18]:
# df2['DIMDATEKEY'] = df2['DIMDATEKEY'].map(lambda a: num_to_date(a))
df2.head()

,DIMDATEKEY,SOMMERSAULT,SOMMERSAULT_MOBILE,CAREER_PAGES,SOMMERSAULT_IOS
0,2018-05-23,1093907,638889,14831,5
1,2018-05-24,1363674,576457,13422,10
2,2018-05-25,1110145,493673,10252,6
3,2018-05-26,1245250,443766,7776,11
4,2018-05-27,1121762,461590,7427,7


In [21]:
df2['SOMMERSAULT'].tolist()

[1093907,
 1363674,
 1110145,
 1245250,
 1121762,
 1046010,
 1422652,
 1497344,
 1529841,
 1250152,
 1395436,
 1165614,
 1652922,
 580187]

In [88]:
dates = df2['DIMDATEKEY'].tolist()

trace1 = go.Bar(
    x=dates,
    y=df2['SOMMERSAULT'].tolist(),
    name='Sommersault',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)

trace2 = go.Bar(
    x=dates,
    y=df2['SOMMERSAULT_MOBILE'].tolist(),
    name='Sommersault Mobile',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)

trace3 = go.Bar(
    x=dates,
    y=df2['CAREER_PAGES'].tolist(),
    name='Career Pages',
    marker=dict(
        color='rgb(1, 1, 1)'
    )
)
trace4 = go.Bar(
    x=dates,
    y=df2['SOMMERSAULT_IOS'].tolist(),
    name='Sommersault iOS',
    marker=dict(
        color='rgb(20, 200, 50)'
    )
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    title='Keyword Searches per Platform per Day',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Keyword Searches',
        titlefont=dict(
            size=18,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.175,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.175,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Keyword searches per platform per day analysis')

In [81]:
# SEARCH-TYPE ANALYSIS (KW, LOC, KW+LOC)

date_from = input("The starting date of the analysis (YYYYMMDD): ")

query3_p1 = """
select DIMDATEKEY,
    count(*) as total_valid_searches,
    count(case when ((REGEXP_COUNT(QUERYSTRING, '(=)', 1) = 1 and REGEXP_COUNT(QUERYSTRING, '(w=)', 1) = 1)
                      or (REGEXP_COUNT(QUERYSTRING, '(=)', 1) < 3 and REGEXP_COUNT(QUERYSTRING, '(l=)', 1) = 1)
                      or (REGEXP_COUNT(QUERYSTRING, '(=)', 1) < 3 and REGEXP_COUNT(QUERYSTRING, '(s=)', 1) = 1))
                      then 1 end) as location_only_search,     
    count(case when (REGEXP_COUNT(QUERYSTRING, '(w=)', 1) = 0
                     and REGEXP_COUNT(QUERYSTRING, '(l=)', 1) = 0
                     and REGEXP_COUNT(QUERYSTRING, '(s=)', 1) = 0)
                     then 1 end) as keyword_only_search
from EVENT.EVENTS where EVENTTYPE='Search' and Placement = 'Keyword Search' and DIMDATEKEY >=
"""

query3_p2 = """
 and QUERYSTRING is not NULL
group by DIMDATEKEY
order by DIMDATEKEY;
"""

query3 = query3_p1 + str(date_from) + query3_p2

col_names3 = ['DIMDATEKEY', 'TOTAL_VALID_SEARCHES', 'KEYWORD_ONLY_SEARCH', 'LOCATION_ONLY_SEARCH']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query3) as results:
            for r in results:
                df3 = pd.concat([pd.DataFrame([r], columns=col_names3) for r in results],
                           ignore_index=True)
            

In [84]:
# df3['KEYWORD_AND_LOCATION_SEARCHES'] = df3['TOTAL_VALID_SEARCHES'] - df3['KEYWORD_ONLY_SEARCH'] \
#                                         - df3['LOCATION_ONLY_SEARCH']
df3['DIMDATEKEY'] = df3['DIMDATEKEY'].map(lambda a: num_to_date(a))

df3.head()

,DIMDATEKEY,TOTAL_VALID_SEARCHES,KEYWORD_ONLY_SEARCH,LOCATION_ONLY_SEARCH,KEYWORD_AND_LOCATION_SEARCHES
0,2018-05-24,1936212,140287,1086266,709659
1,2018-05-25,1599555,99300,907463,592792
2,2018-05-26,1685435,119576,944650,621209
3,2018-05-27,1578382,83871,897718,596793
4,2018-05-28,1574320,183850,724963,665507


In [87]:
## LOOK AT THE FIRST SEARCH, MAYBE GROUP BY SESSION
keyword_only = go.Scatter(
    x=df3['DIMDATEKEY'].tolist(),
    y=df3['KEYWORD_ONLY_SEARCH'].tolist(),
    name='Keyword only'
)

location_only = go.Scatter(
    x=df3['DIMDATEKEY'].tolist(),
    y=df3['LOCATION_ONLY_SEARCH'].tolist(),
    name='Location only'
)

keyword_location = go.Scatter(
    x=df3['DIMDATEKEY'].tolist(),
    y=df3['KEYWORD_AND_LOCATION_SEARCHES'].tolist(),
    name='Keyword and Location'
)

searches = [keyword_only, location_only, keyword_location]

layout = go.Layout(
    title='Search Type Analysis',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Number of Searches',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=searches, layout=layout)

py.iplot(fig, filename = 'Keyword seach type analysis')

In [89]:
# REGISTERED VS UNREGISTERED USERS

date_from = input("The starting date of the analysis (YYYYMMDD): ")

query4_p1 = """
select DIMDATEKEY,
    count(case when JOBSEEKERID is not NULL then 1 end) as registered_users,
    count(case when JOBSEEKERID is NULL then 1 end) as unregistered_users
from EVENT.EVENTS 
where EVENTTYPE='Search' and DIMDATEKEY >=
"""

query4_p2 = """
group by DIMDATEKEY
order by DIMDATEKEY;
"""

query4 = query4_p1 + str(date_from) + query4_p2

In [90]:
col_names4 = ['DIMDATEKEY', 'REGISTERED_USERS', 'UNREGISTERED_USERS']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query4) as results:
            df4 = pd.concat([pd.DataFrame([r], columns=col_names4) for r in results],
                           ignore_index=True)

In [91]:
df4['DIMDATEKEY'] = df4['DIMDATEKEY'].map(lambda a: num_to_date(a))
df4.head()

,DIMDATEKEY,REGISTERED_USERS,UNREGISTERED_USERS
0,2018-03-02,742371,1643617
1,2018-03-03,647973,1713635
2,2018-03-04,740295,1597394
3,2018-03-05,1051300,2344263
4,2018-03-06,1027027,2059672


In [93]:
registered_users = go.Scatter(
    x=df4['DIMDATEKEY'].tolist(),
    y=df4['REGISTERED_USERS'].tolist(),
    name='Registered Users'
)

unregistered_users = go.Scatter(
    x=df4['DIMDATEKEY'].tolist(),
    y=df4['UNREGISTERED_USERS'].tolist(),
    name='Unregistered Users'
)

users = [registered_users, unregistered_users]

layout = go.Layout(
    title='Volume of User Registration',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Number of User Events',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=users, layout=layout)

py.iplot(fig, filename = 'User registration search analysis')

In [18]:
# DEMAND/SEARCH MAP

date_of = input("The date of the searches (YYYYMMDD): ")

query5_p1 = """
select FIPSCODE, count(*) as NUM_SEARCHES from
(select postal.POSTINGID as POSTINGID, postal_to_fips.AREAFIPSCODE as FIPSCODE
from PROD_SAJ_SHARE.DBO.DIMLOCATION as postal_to_fips
right join (select first_posting.POSTINGID, posting_to_postal.POSTALCODE
            from (select regexp_substr(POSTINGID, '([0-9]+)', 1, 1) as POSTINGID
                  from EVENT.EVENTS 
                  where EVENTTYPE='Search' and PLACEMENT = 'Keyword Search' and DIMDATEKEY = 
"""

query5_p2 = """
                  and POSTINGID is not NULL and (REGEXP_COUNT(QUERYSTRING, '(w=)', 1) <> 0 
                  or REGEXP_COUNT(QUERYSTRING, '(l=)', 1) <> 0 or REGEXP_COUNT(QUERYSTRING, '(s=)', 1) <> 0)) 
            as first_posting
            left join PROD_SAJ_SHARE.CUSTOMER.DIMJOBPOSTING_VIEW as posting_to_postal
            on first_posting.POSTINGID = posting_to_postal.POSTINGID) as postal
on postal.POSTALCODE = postal_to_fips.POSTALCODE)
group by 1
order by 2 desc;"""

query5 = query5_p1 + str(date_of) + query5_p2

col_names5 = ['FIPSCODE', 'NUM_SEARCHES']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query5) as results:
            df5 = pd.concat([pd.DataFrame([r], columns=col_names5) for r in results],
                           ignore_index=True)

The date of the searches (YYYYMMDD): 20180611


In [19]:
df5.head()
print('Initial length:', len(df5))
for i in range(len(df5)):
    if((pd.isna(df5['FIPSCODE'])[i]) == True):
        print('Null FIPS Code in row', i)
        
# print(df5.iloc[194])
df5 = df5.dropna()
print('Final length:', len(df5))
df5 = df5.reset_index(drop=True)
df5.describe()

Initial length: 950
Null FIPS Code in row 191
Final length: 949


,NUM_SEARCHES
count,949.000000
mean,1918.869336
std,7214.763711
min,3.000000
25%,158.000000
50%,366.000000
75%,1123.000000
max,118695.000000


In [20]:
df5['FIPSCODE_STRING'] = df5['FIPSCODE'].apply(lambda x: str(x))

df5['FIPSCODE'] = df5['FIPSCODE'].astype('float64', copy=False)

df5['COUNTY_ARRAY'] = [list() for x in range(len(df5))]

msa_to_fips = pd.read_csv('cbsa2fipsxw.csv')

In [41]:
msa_to_fips.head()

,cbsacode,metrodivisioncode,csacode,cbsatitle,metropolitanmicropolitanstatis,metropolitandivisiontitle,csatitle,countycountyequivalent,statename,fipsstatecode,fipscountycode,centraloutlyingcounty
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33860.0,NaN,NaN,"Montgomery, AL",Metropolitan Statistical Area,NaN,NaN,Autauga County,Alabama,1.0,1.0,Central
2,19300.0,NaN,380.0,"Daphne-Fairhope-Foley, AL",Metropolitan Statistical Area,NaN,"Mobile-Daphne-Fairhope, AL",Baldwin County,Alabama,1.0,3.0,Central
3,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Bibb County,Alabama,1.0,7.0,Outlying
4,13820.0,NaN,142.0,"Birmingham-Hoover, AL",Metropolitan Statistical Area,NaN,"Birmingham-Hoover-Talladega, AL",Blount County,Alabama,1.0,9.0,Outlying


In [61]:
counties = dict()

for i in range(len(df5)):
    msa = df5.at[i, 'FIPSCODE']
    for j in range(len(msa_to_fips)):
        if msa_to_fips.at[j, 'cbsacode'] == msa:
            state = str(int(msa_to_fips.at[j, 'fipsstatecode']))
            county = int(msa_to_fips.at[j, 'fipscountycode'])
            if county < 10:
                county = '00' + str(county)
            elif county >= 10 and county < 100:
                county = '0' + str(county)
            else:
                county = str(county)
            fips = state + county
            
            if fips in counties.keys():
                counties[fips] += df5.at[i, 'NUM_SEARCHES']
            else:
                counties[fips] = df5.at[i, 'NUM_SEARCHES']


In [22]:
fips = []
values = []

for k in counties.keys():
    fips.append(k)
    values.append(counties[k])
    

fig = ff.create_choropleth(fips=fips, values=values, 
                            binning_endpoints=[200, 500, 1000, 10000, 20000, 50000],
                           title='Number of Searches per MSA', 
                           legend_title='Number of Searches')
py.iplot(fig, filename='Demand Map (Number of Searches per MSA)', sort=True)

/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




The draw time for this plot will be slow for clients without much RAM.


/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [24]:
# SUPPLY/POSTINGS MAP

date_of = input("The date of the postings (YYYY-MM-DD): ")

query6_p1 = """
select FIPSCODE, count(*) as NUM_POSTINGS
from 
(select posting_postal.POSTINGID as POSTINGID, postal_to_fips.AREAFIPSCODE as FIPSCODE
from PROD_SAJ_SHARE.DBO.DIMLOCATION postal_to_fips
right join (select POSTINGID, POSTALCODE
            from CUSTOMER.DIMJOBPOSTING_VIEW
            where DEACTIVATIONDATE > to_timestamp('
"""

query6_p2 = """
             00:00:00')
            and CREATEDATE < to_timestamp('
"""

query6_p3 = """
             00:00:00')) as posting_postal
on posting_postal.POSTALCODE = postal_to_fips.POSTALCODE)
group by 1
order by 2 desc;

"""

query6 = query6_p1 + str(date_of) + query6_p2 + str(date_of) + query6_p3

col_names6 = ['FIPSCODE', 'NUM_POSTINGS']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query6) as results:
            df6 = pd.concat([pd.DataFrame([r], columns=col_names6) for r in results],
                           ignore_index=True)

The date of the postings (YYYY-MM-DD): 2018-06-11


In [25]:
df6.head()
print('Initial length:', len(df6))
for i in range(len(df6)):
    if((pd.isna(df6['FIPSCODE'])[i]) == True):
        print('Null FIPS Code in row', i)
        
df6 = df6.dropna()
print('Final length:', len(df6))
df6 = df6.reset_index(drop=True)
df6.describe()

Initial length: 950
Null FIPS Code in row 236
Final length: 949


,NUM_POSTINGS
count,949.000000
mean,1208.229715
std,3829.324550
min,1.000000
25%,128.000000
50%,281.000000
75%,674.000000
max,50626.000000


In [35]:
df5.describe()

,FIPSCODE,NUM_SEARCHES
count,949.000000,949.000000
mean,30744.583772,1918.869336
std,13106.563916,7214.763711
min,0.000000,3.000000
25%,20140.000000,158.000000
50%,30280.000000,366.000000
75%,40140.000000,1123.000000
max,79600.000000,118695.000000


In [62]:
df6['FIPSCODE_STRING'] = df6['FIPSCODE'].apply(lambda x: str(x))

df6['FIPSCODE'] = df6['FIPSCODE'].astype('float64', copy=False)

msa_to_fips = pd.read_csv('cbsa2fipsxw.csv')

counties_posting = dict()

for i in range(len(df6)):
    msa = df6.at[i, 'FIPSCODE']
    for j in range(len(msa_to_fips)):
        if msa_to_fips.at[j, 'cbsacode'] == msa:
            state = str(int(msa_to_fips.at[j, 'fipsstatecode']))
            county = int(msa_to_fips.at[j, 'fipscountycode'])
            if county < 10:
                county = '00' + str(county)
            elif county >= 10 and county < 100:
                county = '0' + str(county)
            else:
                county = str(county)
            fips = state + county
            
            if fips in counties_posting.keys():
                counties_posting[fips] += df6.at[i, 'NUM_POSTINGS']
            else:
                counties_posting[fips] = df6.at[i, 'NUM_POSTINGS']
                
fips = []
values = []

for k in counties_posting.keys():
    fips.append(k)
    values.append(counties_posting[k])
    

fig = ff.create_choropleth(fips=fips, values=values, 
                            binning_endpoints=[200, 500, 1000, 10000, 20000, 40000],
                           title='Number of Postings per MSA', 
                           legend_title='Number of Postings')
py.iplot(fig, filename='Supply Map (Number of Postings per county)', sort=True)

/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




The draw time for this plot will be slow for clients without much RAM.


/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [69]:
# SUPPLY/DEMAND RATIO MAP

counties_ratio = dict()

counties_keys = counties.keys()

for k in counties_posting.keys():
    if k in counties_keys:
        counties_ratio[k] = (counties[k] / counties_posting[k])
    else:
        del counties_ratio[k]
        
fips = []
values = []

for k in counties_ratio.keys():
    fips.append(k)
    values.append(counties_ratio[k])
    
mean = np.mean(values)

for k in counties_ratio.keys():
    counties_ratio[k] = counties_ratio[k] / mean
    
# np.min(list(counties_ratio.values()))

fig = ff.create_choropleth(fips=fips, values=values, 
                            binning_endpoints=[0.25, 0.5, 1, 3],
                           title='Map of Ratio of Searches to Postings per MSA', 
                           legend_title='Ratio of Searches to Postings')
py.iplot(fig, filename='Map of Ratio of Searches to Postings (per MSA)', sort=True)

/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




The draw time for this plot will be slow for clients without much RAM.


/Users/nazih.bissat/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [12]:
# BOUNCE RATE ANALYSIS

date_from = input("The date to start analysis from (YYYYMMDD): ")

query_google_p1 = """
select final.DIMDATEKEY,
    count(case
          when NUM_PAGE_VIEW = NUM_OPENED_WINDOWS_GOOGLE then NUM_PAGE_VIEW
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then 2 * NUM_OPENED_WINDOWS_GOOGLE - NUM_PAGE_VIEW
          end) as BOUNCE_SESSION_GOOGLE,
    count(case
          when NUM_OPENED_WINDOWS_GOOGLE = 1 and NUM_PAGE_VIEW > 1 then 1
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then NUM_PAGE_VIEW - NUM_OPENED_WINDOWS_GOOGLE
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE <= NUM_PAGE_VIEW)) then NUM_OPENED_WINDOWS_GOOGLE
          end) as NO_BOUNCE_SESSION_GOOGLE
from (select SESSIONID, DIMDATEKEY,
        count(case when EVENTTYPE = 'Page View' then 1 end) as NUM_PAGE_VIEW,
        count(case when contains(REFERRINGURL, 'google.com') 
              and EVENTTYPE = 'Page View' then 1 end) as NUM_OPENED_WINDOWS_GOOGLE
from (select a.SESSIONID, a.DIMDATEKEY, e.EVENTTYPE, e.PLACEMENT, e.REFERRINGURL, e.POSTINGID, e.EVENTTIME
      from (select SESSIONID, DIMDATEKEY
            from EVENT.EVENTS
            where DIMDATEKEY >=
"""


query_google_p2 = """
 and SESSIONID is not NULL
            and SESSIONID <> '00000000-0000-0000-0000-000000000000' and
            contains(REFERRINGURL, 'google.com') 
            and (EVENTTYPE = 'Posting Detail Impression' or EVENTTYPE = 'Search')) as a
       left join (select * from EVENT.EVENTS
                  where DIMDATEKEY >= 
"""

query_google_p3 = """
 and SESSIONID is not NULL
                  and SESSIONID <> '00000000-0000-0000-0000-000000000000') as e
      on a.SESSIONID = e.SESSIONID) as j
group by SESSIONID, DIMDATEKEY) as final
group by 1
order by 1;
"""

query_google = query_google_p1 + str(date_from) + query_google_p2 + str(date_from) + query_google_p3

col_names_br = ['DIMDATEKEY', 'NO_BOUNCE_SESSION_GOOGLE', 'BOUNCE_SESSION_GOOGLE']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query_google) as results:
            df_br = pd.concat([pd.DataFrame([r], columns=col_names_br) for r in results],
                           ignore_index=True)

df_br['BOUNCE_RATE_GOOGLE'] = 100 * df_br['BOUNCE_SESSION_GOOGLE'] / (df_br['BOUNCE_SESSION_GOOGLE']
                                                                      + df_br['NO_BOUNCE_SESSION_GOOGLE'])

df_br.head()

The date to start analysis from (YYYYMMDD): 20180522


,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GOOGLE
0,20180522,106465,62685,37.058824
1,20180523,104091,59242,36.270686
2,20180524,94682,55738,37.054913
3,20180525,80796,46913,36.734294
4,20180526,65229,36735,36.027421


In [41]:
query_non_google_p1 = """
select a.DIMDATEKEY,
    count(case when NUM_PAGE_VIEW > 1 then 1 end) as NO_BOUNCE_SESSION_NON_GOOGLE,
    count(case when NUM_PAGE_VIEW <= 1 then 1 end) as BOUNCE_SESSION_NON_GOOGLE
from (select SESSIONID, DIMDATEKEY,
        count(case when EVENTTYPE = 'Page View' then 1 end) as NUM_PAGE_VIEW
      from EVENT.EVENTS 
      where DIMDATEKEY >= 
"""

query_non_google_p2 = """
      and SESSIONID is not NULL 
      and SESSIONID<>'00000000-0000-0000-0000-000000000000' and not contains(REFERRINGURL, 'google.com')
      and (EVENTTYPE='Search' or EVENTTYPE='Posting Detail Impression' or EVENTTYPE = 'Page View')
      group by SESSIONID, DIMDATEKEY) as a
group by 1
order by 1;
"""

query_non_google = query_non_google_p1 + str(date_from) + query_non_google_p2

col_names_br_non_google = ['DIMDATEKEY', 'NO_BOUNCE_SESSION_NON_GOOGLE', 'BOUNCE_SESSION_NON_GOOGLE']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query_non_google) as results:
            df_non_google = pd.concat([pd.DataFrame([r], columns=col_names_br_non_google) for r in results],
                               ignore_index=True)
            
df_non_google['BOUNCE_RATE_NON_GOOGLE'] = \
100 * df_non_google['BOUNCE_SESSION_NON_GOOGLE'] / (df_non_google['BOUNCE_SESSION_NON_GOOGLE']
                                                    + df_non_google['NO_BOUNCE_SESSION_NON_GOOGLE'])

df_non_google.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_NON_GOOGLE,BOUNCE_SESSION_NON_GOOGLE,BOUNCE_RATE_NON_GOOGLE
0,20180312,126419,82649,39.532114
1,20180313,128701,82748,39.133786
2,20180314,112235,91331,44.865547
3,20180315,122266,67349,35.518814
4,20180316,110915,63176,36.289067


In [42]:
df_br_joined = pd.concat([df_br, df_non_google.drop(['DIMDATEKEY'], axis = 1)], axis=1, sort=False)
df_br_joined['DIMDATEKEY'] = df_br_joined['DIMDATEKEY'].map(lambda a: num_to_date(a))
df_br_joined.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GOOGLE,NO_BOUNCE_SESSION_NON_GOOGLE,BOUNCE_SESSION_NON_GOOGLE,BOUNCE_RATE_NON_GOOGLE
0,2018-03-12,83678,52890,38.727960,126419,82649,39.532114
1,2018-03-13,80458,51662,39.102331,128701,82748,39.133786
2,2018-03-14,77016,49045,38.905768,112235,91331,44.865547
3,2018-03-15,72268,43860,37.768669,122266,67349,35.518814
4,2018-03-16,60354,37021,38.018999,110915,63176,36.289067


In [43]:
df_br_joined['TOTAL_BOUNCE_RATE'] = \
100 * (df_br_joined['BOUNCE_SESSION_GOOGLE'] + df_br_joined['BOUNCE_SESSION_NON_GOOGLE']) / \
(df_br_joined['BOUNCE_SESSION_GOOGLE'] + df_br_joined['BOUNCE_SESSION_NON_GOOGLE'] + \
 df_br_joined['NO_BOUNCE_SESSION_GOOGLE'] + df_br_joined['NO_BOUNCE_SESSION_NON_GOOGLE'])

In [44]:
# REMOVE BOTS (BOTTRAFFIC = 0 and potentially other stuff)

br_google = go.Scatter(
    x=df_br_joined['DIMDATEKEY'].tolist(),
    y=df_br_joined['BOUNCE_RATE_GOOGLE'].tolist(),
    name='Google Bounce Rate'
)

br_non_google = go.Scatter(
    x=df_br_joined['DIMDATEKEY'].tolist(),
    y=df_br_joined['BOUNCE_RATE_NON_GOOGLE'].tolist(),
    name='Other Bounce Rate'
)

br_total = go.Scatter(
    x=df_br_joined['DIMDATEKEY'].tolist(),
    y=df_br_joined['TOTAL_BOUNCE_RATE'].tolist(),
    name='Total Bounce Rate'
)

br = [br_google, br_non_google, br_total]

layout = go.Layout(
    title='Bounce Rate Analysis (Session-Level)',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Bounce Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=br, layout=layout)

py.iplot(fig, filename = 'Bounce rate analysis')

In [13]:
# LOWER/UPPER BOUNDS (RANGE) FOR GOOGLE BOUNCE RATE

query_up_google_p1 = """
select final.DIMDATEKEY,
    count(case
          when NUM_PAGE_VIEW = NUM_OPENED_WINDOWS_GOOGLE then NUM_PAGE_VIEW
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE
                and (2 * NUM_OPENED_WINDOWS_GOOGLE >= NUM_PAGE_VIEW)) then NUM_OPENED_WINDOWS_GOOGLE - 1
          end) as BOUNCE_SESSION_GOOGLE,
    count(case
          when NUM_OPENED_WINDOWS_GOOGLE = 1 and NUM_PAGE_VIEW > 1 then 1
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE >= NUM_PAGE_VIEW)) then 1
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE < NUM_PAGE_VIEW)) then NUM_OPENED_WINDOWS_GOOGLE
          end) as NO_BOUNCE_SESSION_GOOGLE
from (select SESSIONID, DIMDATEKEY,
        count(case when EVENTTYPE = 'Page View' then 1 end) as NUM_PAGE_VIEW,
        count(case when contains(REFERRINGURL, 'google.com') 
              and EVENTTYPE = 'Page View' then 1 end) as NUM_OPENED_WINDOWS_GOOGLE
from (select a.SESSIONID, a.DIMDATEKEY, e.EVENTTYPE, e.PLACEMENT, e.REFERRINGURL, e.POSTINGID, e.EVENTTIME
      from (select SESSIONID, DIMDATEKEY
            from EVENT.EVENTS
            where DIMDATEKEY >=
"""


query_up_google_p2 = """
 and SESSIONID is not NULL
            and SESSIONID <> '00000000-0000-0000-0000-000000000000' and
            contains(REFERRINGURL, 'google.com') 
            and (EVENTTYPE = 'Posting Detail Impression' or EVENTTYPE = 'Search')) as a
       left join (select * from EVENT.EVENTS
                  where DIMDATEKEY >= 
"""

query_up_google_p3 = """
 and SESSIONID is not NULL
                  and SESSIONID <> '00000000-0000-0000-0000-000000000000') as e
      on a.SESSIONID = e.SESSIONID) as j
group by SESSIONID, DIMDATEKEY) as final
group by 1
order by 1;
"""

query_up_google = query_up_google_p1 + str(date_from) + query_up_google_p2 + str(date_from) + query_up_google_p3

col_names_br_up = ['DIMDATEKEY', 'NO_BOUNCE_SESSION_GOOGLE', 'BOUNCE_SESSION_GOOGLE']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query_up_google) as results:
            df_br_up = pd.concat([pd.DataFrame([r], columns=col_names_br_up) for r in results],
                               ignore_index=True)

df_br_up['BOUNCE_RATE_GOOGLE_UPPER'] = 100 * df_br_up['BOUNCE_SESSION_GOOGLE'] / (df_br_up['BOUNCE_SESSION_GOOGLE']
                                                                      + df_br_up['NO_BOUNCE_SESSION_GOOGLE'])

df_br_up.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GOOGLE_UPPER
0,20180522,106465,68442,39.130509
1,20180523,104091,64692,38.328505
2,20180524,94682,60610,39.029699
3,20180525,80796,50897,38.648220
4,20180526,65229,40029,38.029413


In [14]:
df_br_bounds = pd.concat([df_br, df_br_up.drop(['DIMDATEKEY'], axis = 1)], axis=1, sort=False)
df_br_bounds['DIMDATEKEY'] = df_br_bounds['DIMDATEKEY'].map(lambda a: num_to_date(a))
df_br_bounds.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GOOGLE,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GOOGLE_UPPER
0,2018-05-22,106465,62685,37.058824,106465,68442,39.130509
1,2018-05-23,104091,59242,36.270686,104091,64692,38.328505
2,2018-05-24,94682,55738,37.054913,94682,60610,39.029699
3,2018-05-25,80796,46913,36.734294,80796,50897,38.648220
4,2018-05-26,65229,36735,36.027421,65229,40029,38.029413


In [43]:
trace0 = go.Scatter(
    x=df_br_bounds['DIMDATEKEY'].tolist(),
    y=df_br_bounds['BOUNCE_RATE_GOOGLE'].tolist(),
    name='Lower bound',
    fill= None,
    mode='lines',
    line=dict(
        color='rgb(143, 19, 131)',
    )
)
trace1 = go.Scatter(
    x=df_br_bounds['DIMDATEKEY'].tolist(),
    y=df_br_bounds['BOUNCE_RATE_GOOGLE_UPPER'].tolist(),
    name='Upper bound',
    fill='tonexty',
    mode='lines',
    line=dict(
        color='rgb(143, 19, 131)',
    )
)

layout = go.Layout(
    title='Range of Google Bounce Rate (Session-Level)',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Bounce Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

data = [trace0, trace1]

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename ='Lower vs. Upper Bound for Google Bounce Rate')

In [44]:
# GOOGLE SEARCH VS GOOGLE JOBS WIDGET BOUNCE RATE ANALYSIS

date_from = input("The date to start analysis from (YYYYMMDD): ")

query_gjobs_p1 = """
select final.DIMDATEKEY,
    count(case
          when NUM_PAGE_VIEW = NUM_OPENED_WINDOWS_GOOGLE then NUM_PAGE_VIEW
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then 2 * NUM_OPENED_WINDOWS_GOOGLE - NUM_PAGE_VIEW
          end) as BOUNCE_SESSION_GOOGLE,
    count(case
          when NUM_OPENED_WINDOWS_GOOGLE = 1 and NUM_PAGE_VIEW > 1 then 1
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then NUM_PAGE_VIEW - NUM_OPENED_WINDOWS_GOOGLE
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE <= NUM_PAGE_VIEW)) then NUM_OPENED_WINDOWS_GOOGLE
          end) as NO_BOUNCE_SESSION_GOOGLE
from (select SESSIONID, DIMDATEKEY,
        count(case when EVENTTYPE = 'Page View' then 1 end) as NUM_PAGE_VIEW,
        count(case when contains(REFERRINGURL, 'google.com') 
              and EVENTTYPE = 'Page View' then 1 end) as NUM_OPENED_WINDOWS_GOOGLE
from (select a.SESSIONID, a.DIMDATEKEY, e.EVENTTYPE, e.PLACEMENT, e.REFERRINGURL, e.POSTINGID, e.EVENTTIME
      from (select SESSIONID, DIMDATEKEY
            from EVENT.EVENTS
            where DIMDATEKEY >=
"""


query_gjobs_p2 = """
 and SESSIONID is not NULL
            and SESSIONID <> '00000000-0000-0000-0000-000000000000' and
            contains(REFERRINGURL, 'google.com') 
            and (EVENTTYPE = 'Posting Detail Impression')) as a
       left join (select * from EVENT.EVENTS
                  where DIMDATEKEY >= 
"""

query_gjobs_p3 = """
 and SESSIONID is not NULL
                  and SESSIONID <> '00000000-0000-0000-0000-000000000000') as e
      on a.SESSIONID = e.SESSIONID) as j
group by SESSIONID, DIMDATEKEY) as final
group by 1
order by 1;
"""

query_gjobs = query_gjobs_p1 + str(date_from) + query_gjobs_p2 + str(date_from) + query_gjobs_p3

col_names_br = ['DIMDATEKEY', 'NO_BOUNCE_SESSION_GOOGLE', 'BOUNCE_SESSION_GOOGLE']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query_gjobs) as results:
            df_gjobs = pd.concat([pd.DataFrame([r], columns=col_names_br) for r in results],
                           ignore_index=True)

df_gjobs['BOUNCE_RATE_GJOBS'] = 100 * df_gjobs['BOUNCE_SESSION_GOOGLE'] / \
                            (df_gjobs['BOUNCE_SESSION_GOOGLE'] + \
                             df_gjobs['NO_BOUNCE_SESSION_GOOGLE'])

df_gjobs.head()

The date to start analysis from (YYYYMMDD): 20180312


,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GJOBS
0,20180312,10195,4787,31.951675
1,20180313,10135,4907,32.621992
2,20180314,10205,4779,31.894020
3,20180315,9625,4477,31.747270
4,20180316,7807,3768,32.552916


In [53]:
query_gsearch_p1 = """
select final.DIMDATEKEY,
    count(case
          when NUM_PAGE_VIEW = NUM_OPENED_WINDOWS_GOOGLE then NUM_PAGE_VIEW
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then 2 * NUM_OPENED_WINDOWS_GOOGLE - NUM_PAGE_VIEW
          end) as BOUNCE_SESSION_GOOGLE,
    count(case
          when NUM_OPENED_WINDOWS_GOOGLE = 1 and NUM_PAGE_VIEW > 1 then 1
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE > NUM_PAGE_VIEW)) then NUM_PAGE_VIEW - NUM_OPENED_WINDOWS_GOOGLE
          when (NUM_OPENED_WINDOWS_GOOGLE > 1 
                and NUM_PAGE_VIEW > NUM_OPENED_WINDOWS_GOOGLE 
                and (2 * NUM_OPENED_WINDOWS_GOOGLE <= NUM_PAGE_VIEW)) then NUM_OPENED_WINDOWS_GOOGLE
          end) as NO_BOUNCE_SESSION_GOOGLE
from (select SESSIONID, DIMDATEKEY,
        count(case when EVENTTYPE = 'Page View' then 1 end) as NUM_PAGE_VIEW,
        count(case when contains(REFERRINGURL, 'google.com') 
              and EVENTTYPE = 'Page View' then 1 end) as NUM_OPENED_WINDOWS_GOOGLE
from (select a.SESSIONID, a.DIMDATEKEY, e.EVENTTYPE, e.PLACEMENT, e.REFERRINGURL, e.POSTINGID, e.EVENTTIME
      from (select SESSIONID, DIMDATEKEY
            from EVENT.EVENTS
            where DIMDATEKEY >=
"""


query_gsearch_p2 = """
 and SESSIONID is not NULL
            and SESSIONID <> '00000000-0000-0000-0000-000000000000' and
            contains(REFERRINGURL, 'google.com') 
            and (EVENTTYPE = 'Search')) as a
       left join (select * from EVENT.EVENTS
                  where DIMDATEKEY >= 
"""

query_gsearch_p3 = """
 and SESSIONID is not NULL
                  and SESSIONID <> '00000000-0000-0000-0000-000000000000') as e
      on a.SESSIONID = e.SESSIONID) as j
group by SESSIONID, DIMDATEKEY) as final
group by 1
order by 1;
"""

query_gsearch = query_gsearch_p1 + str(date_from) + query_gsearch_p2 + str(date_from) + \
                query_gsearch_p3

col_names_br = ['DIMDATEKEY', 'NO_BOUNCE_SESSION_GOOGLE', 'BOUNCE_SESSION_GOOGLE']

with snowflake.connector.connect(** CONNECTION_PARAMS) as ctx:
    with ctx.cursor(DictCursor) as cs:
        with cs.execute(query_gsearch) as results:
            df_gsearch = pd.concat([pd.DataFrame([r], columns=col_names_br) for r in results],
                           ignore_index=True)

df_gsearch['BOUNCE_RATE_GSEARCH'] = 100 * df_gsearch['BOUNCE_SESSION_GOOGLE'] / \
                                    (df_gsearch['BOUNCE_SESSION_GOOGLE'] + \
                                     df_gsearch['NO_BOUNCE_SESSION_GOOGLE'])

df_gsearch.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GSEARCH
0,20180312,78025,48757,38.457352
1,20180313,74597,47402,38.854417
2,20180314,70807,44854,38.780574
3,20180315,66408,39964,37.570037
4,20180316,55838,33741,37.666194


In [50]:
df_gjobs['BOUNCE_RATE_GJOBS'] = 100 * df_gjobs['BOUNCE_SESSION_GOOGLE'] / \
                            (df_gjobs['BOUNCE_SESSION_GOOGLE'] + \
                             df_gjobs['NO_BOUNCE_SESSION_GOOGLE'])
df_gjobs.head()

,DIMDATEKEY,NO_BOUNCE_SESSION_GOOGLE,BOUNCE_SESSION_GOOGLE,BOUNCE_RATE_GJOBS
0,20180312,10195,4787,31.951675
1,20180313,10135,4907,32.621992
2,20180314,10205,4779,31.894020
3,20180315,9625,4477,31.747270
4,20180316,7807,3768,32.552916


In [54]:
# df_br_joined = pd.concat([df_br, df_non_google.drop(['DIMDATEKEY'], axis = 1)], axis=1, sort=False)
# df_br_joined['DIMDATEKEY'] = df_br_joined['DIMDATEKEY'].map(lambda a: num_to_date(a))
# df_br_joined.head()

br_gjobs = go.Scatter(
    x=df_gjobs['DIMDATEKEY'].map(lambda a: num_to_date(a)).tolist(),
    y=df_gjobs['BOUNCE_RATE_GJOBS'].tolist(),
    name='Google Jobs Widget'
)

br_gsearch = go.Scatter(
    x=df_gsearch['DIMDATEKEY'].map(lambda a: num_to_date(a)).tolist(),
    y=df_gsearch['BOUNCE_RATE_GSEARCH'].tolist(),
    name='Google Search'
)

br = [br_gjobs, br_gsearch]

layout = go.Layout(
    title='Google Bounce Rate Analysis (Session-Level)',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Bounce Rate (%)',
        titlefont=dict(
            size=14,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=br, layout=layout)

py.iplot(fig, filename = 'Google bounce rate analysis')